## Libraries



In [ ]:
# Check if libraries are already installed > otherwise install it
if(!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager",repos = "http://cran.us.r-project.org")
if(!"knitr" %in% installed.packages()) BiocManager::install("knitr")
if(!"jsonlite" %in% installed.packages()) BiocManager::install("jsonlite")
if(!"dplyr" %in% installed.packages()) BiocManager::install("dplyr")
if(!"clusterProfiler" %in% installed.packages()) BiocManager::install("clusterProfiler")
if(!"ggplot2" %in% installed.packages()) BiocManager::install("ggplot2")

# Load packages
library(knitr)
library(jsonlite)
library(dplyr)
library(clusterProfiler)
library(ggplot2)


## Generate HMDB.csv for data annotation

This can be skipped...


In [ ]:
# Sets current path as working file path for all code chuncks
knitr::opts_knit$set(root.dir = ".")

# Read  the metabolomics data file
mbxCount <- read.table(file = "../preprocessing/output/mbxData", sep = '\t', header = TRUE)
# Read metadata file sample labels
metaData <- read.table(file = "../preprocessing/output/mbxMetaData", sep = '\t', stringsAsFactors = TRUE, header = TRUE)

# Read JSON file with all the pathways and the compounds within into R object
PW.to.compound <- fromJSON("data/all_pathways_compounds_dict.json")
# Read JSON file with all the pathways codes and their names within into R object
PW.all.names <- fromJSON("data/ec_pathway_names_dict.json")


## Filtering Steps

We will apply some filtering process to filter out genes in the input data


In [ ]:
# Remove genes which has all zero values for all samples then start DE analysis
mbxCount[is.na(mbxCount)] <- 0
nonzero <- rowSums(mbxCount[,c(3:ncol(mbxCount))]) > 0
mbxCount <- mbxCount[nonzero,]

# Remove rows with more than 50% of 0 values
removeLines <- rowSums(is.na(mbxCount[,3:ncol(mbxCount)]))
fifty_percent <- floor((ncol(mbxCount))/2)
MissingDataCounted <- cbind(mbxCount, removeLines)
NoMissingData <- subset(MissingDataCounted, removeLines <= fifty_percent)
#Remove last column for further processing.
NoMissingData <- subset(NoMissingData, select=-c(removeLines))
NoMissingData[, c(3:ncol(NoMissingData))] <- apply(NoMissingData[, c(3:ncol(NoMissingData))],2, function(x) as.numeric(as.character(x)))

remove(nonzero, removeLines, fifty_percent, MissingDataCounted)


## T-tests



In [ ]:
# Separate CD, UC and nonIBD
metaData.CD <- metaData[metaData$diagnosis=="CD",]
metaData.UC <- metaData[metaData$diagnosis=="UC",]
metaData.nonIBD <- metaData[metaData$diagnosis=="nonIBD",]

# Select metagenomics data from CD, UC and nonIBD
mbxCount.CD <- subset(NoMissingData, select=metaData.CD$External.ID)
mbxCount.UC <- subset(NoMissingData, select=c(1,2, metaData.UC$External.ID))
mbxCount.nonIBD <- subset(NoMissingData, select=c(1,2, metaData.nonIBD$External.ID))

# Normalization using natural logarithm
mbxCount.log.CD <- cbind(mbxCount.CD[,c(1,2)], log2(mbxCount.CD[,3:ncol(mbxCount.CD)]+1))
mbxCount.log.UC <- cbind(mbxCount.UC[,c(1,2)], log2(mbxCount.UC[,3:ncol(mbxCount.UC)]+1))
mbxCount.log.nonIBD <- cbind(mbxCount.nonIBD[,c(1,2)], log2(mbxCount.nonIBD[,3:ncol(mbxCount.nonIBD)]+1))

remove(metaData.CD, metaData.UC, metaData.nonIBD, metaData, mbxCount, NoMissingData, mbxCount.CD, mbxCount.UC, mbxCount.nonIBD)


### T-tests for Crohn's Disease (CD)



In [ ]:
# Compute statistical significance (using t-test)
pValue.CD = NULL # Empty list for the p-values

for(i in 1 : nrow(mbxCount.log.nonIBD)) { # For each metabolite : 
	x = mbxCount.log.nonIBD[i,-(1:2)] # control of metabolite number i
	y = mbxCount.log.CD[i,-(1:2)] # CD of metabolite number i
	
	#x = x[is.finite(x)]
	#y = y[is.finite(y)]
	
	# Compute t-test between the two conditions
	t = t.test(x, y)
	
	# Put the current p-value in the pvalues list
	pValue.CD[i] = t$p.value
}

# Apply Benjamini Hochberg correction to pvalues to correct for multiple testing
#pAdjust.CD = p.adjust(pValue.CD, method="BH", n= nrow(mbxCount.nonIBD))

#print(length(which(pValue.CD<0.05)))
remove(t, x, y, i)


### T-tests for Ulcerative colitis (UC)



In [ ]:
# Compute statistical significance (using t-test)
pValue.UC = NULL # Empty list for the p-values

for(i in 1 : nrow(mbxCount.log.nonIBD)) { # For each gene : 
	x = mbxCount.log.nonIBD[i,-(1:2)] # control of gene number i
	y = mbxCount.log.UC[i,-(1:2)] # UC of gene number i
	
	# Compute t-test between the two conditions
	t = t.test(x, y)
	
	# Put the current p-value in the pvalues list
	pValue.UC[i] = t$p.value
}

# Apply Benjamini Hochberg correction to pvalues to correct for multiple testing
#pAdjust.UC = p.adjust(pValue.UC, method="BH", n= nrow(mbxCount.nonIBD))

#print(length(which(pValue.UC<0.05)))

remove(t, x, y, i)


### Select statistically significant enzymes



In [ ]:
# Select statistically significant enzymes
deg.CD <- mbxCount.log.CD[which(pValue.CD<0.05),]
deg.CD$pValue <- pValue.CD[which(pValue.CD<0.05)]
deg.UC <- mbxCount.log.UC[which(pValue.UC<0.05),]
deg.UC$pValue <- pValue.UC[which(pValue.UC<0.05)]

remove(pValue.CD, pValue.UC)


## Generate mbxData

The following Shinyapp was used to map HMDB IDs to KEGG IDs

https://tabbassidaloii.shinyapps.io/BridgeDb-Shiny/


In [ ]:
# Read KEGG compound to HMDB list
compound.HMDB <- read.table(file = "data/XrefBatch_mapping_BridgeDB-Shiny.csv", sep = ',', header = TRUE)
# Remove ... columns
compound.HMDB <- compound.HMDB[, -c(2,4)]
# Change column names
colnames(compound.HMDB) <- c("HMDB", "compound")

remove(mbxCount.log.CD, mbxCount.log.UC, mbxCount.log.nonIBD)


In [ ]:
# Merge the KEGG compound to the full metabolomics data
deg.CD <- merge(deg.CD, compound.HMDB, by = "HMDB", all.x = TRUE)
#length(which(is.na(deg.CD$compound)))
# remove NAs for compounds
deg.CD <- deg.CD[!is.na(deg.CD$compound), ]
# Relocate the compound column as first
deg.CD <- deg.CD %>% relocate(compound)

# Merge the KEGG compound to the full metabolomics data
deg.UC <- merge(deg.UC, compound.HMDB, by = "HMDB", all.x = TRUE)
#length(which(is.na(deg.UC$compound)))
# remove NAs for compounds
deg.UC <- deg.UC[!is.na(deg.UC$compound), ]
# Relocate the compound column as first
deg.UC <- deg.UC %>% relocate(compound)

remove(compound.HMDB)


## Calculate the ORA score for each pathway


### ORA score for Ulcerative colitis (UC)


In [ ]:
# Transform PW to compound for further analysis
PW.to.compound <- as.data.frame(t(as.data.frame(sapply(PW.to.compound, rbind))))
PW.to.compound$pathway <- rownames(PW.to.compound)
colnames(PW.to.compound) <- c("compound", "pathway")
PW.to.compound <- PW.to.compound[,c(2,1)]
PW.to.compound$pathway <- sub("\\..*", "", PW.to.compound$pathway)
PW.to.compound$compound <- sub("\\s\\s.*", "", PW.to.compound$compound)

PW.to.name <- data.frame("pathway"=names(PW.all.names), "name"=unlist(PW.all.names))

# Perform Pathway enrichment
res.UC <- enricher(gene=as.factor(unlist(deg.UC)),
                TERM2GENE = PW.to.compound,
                TERM2NAME = PW.to.name)

as.data.frame(res.UC)


### ORA score for Crohn's Disease (CD)



In [ ]:
# Perform Pathway enrichment
res.CD <- enricher(gene=as.factor(unlist(deg.CD)),
                TERM2GENE = PW.to.compound,
                TERM2NAME = PW.to.name)

as.data.frame(res.CD)


## Plot the results in a horizontal bar chart

### Plot for Ulcerative colitis (UC)


In [ ]:
# Horizontal bar plot of significant pathways in UC
res.UC[which(res.UC$p.adjust < 0.05),] %>% 
  ggplot(aes(reorder(as.character(Description), Count), Count)) + 
  geom_col(aes(fill = p.adjust)) + 
  scale_fill_gradient2(low = "#0000ff", 
                       mid = "#ff0000",
                       high= "yellow",
                       midpoint = 0.001) +
  coord_flip() + 
  labs(x = "Pathways (UC)", y = "Enzyme count")


### Plot for Crohn's Disease (CD)



In [ ]:
res.CD[which(res.CD$p.adjust < 0.05),] %>% 
  ggplot(aes(reorder(as.character(Description), Count), Count)) + 
  geom_col(aes(fill = p.adjust)) + 
  scale_fill_gradient2(low = "#0000ff", 
                       mid = "#ff0000",
                       high= "yellow",
                       midpoint = 0.001) +
  coord_flip() + 
  labs(x = "Pathways (UC)", y = "Enzyme count")


## Writing output tables in csv file



In [ ]:
# Write the output in a file
write.table(res.UC, "output/mbxPWdata_UC.csv", sep =",", row.names = FALSE)

# Write the output in a file
write.table(res.CD, "output/mbxPWdata_CD.csv", sep =",", row.names = FALSE)


## Print session info:



In [ ]:
##Print session info:
sessionInfo()



## Creating jupyter notebook files


In [ ]:
#Jupyter Notebook file
if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
library(devtools)
library(rmd2jupyter)
rmd2jupyter("metabolomics_analysis.Rmd")
